# Tutorial 02: Interface Learning (FePt on MgO)

**Goal**: Teach the potential how Fe/Pt atoms interact with the MgO surface.

This tutorial demonstrates:
1.  **Interface Construction**: Building a complex hetero-structure (Cluster on Slab).
2.  **Targeted Learning**: Using the Orchestrator to learn a specific configuration rather than random exploration.
3.  **Periodic Embedding**: Handling non-periodic features in a periodic simulation.



In [ ]:
import os
import shutil
from pathlib import Path
from unittest.mock import patch

import numpy as np
from ase.build import bulk, surface, add_adsorbate
from ase.visualize import view

from mlip_autopipec.core.orchestrator import Orchestrator
from mlip_autopipec.domain_models.config import (
    GlobalConfig,
    AdaptiveGeneratorConfig,
    QEOracleConfig,
    PacemakerTrainerConfig,
    LAMMPSDynamicsConfig,
    StandardValidatorConfig,
    MockGeneratorConfig,
    MockOracleConfig,
    MockTrainerConfig,
    MockDynamicsConfig,
    MockValidatorConfig,
    PhysicsBaselineConfig
)
from mlip_autopipec.domain_models.enums import (
    GeneratorType,
    OracleType,
    TrainerType,
    DynamicsType,
    ValidatorType,
)
from mlip_autopipec.domain_models.structure import Structure
from mlip_autopipec.components.oracle.embedding import embed_cluster



## 1. Setup Environment

Same "Mock vs Real" detection as Tutorial 01.



In [ ]:
def has_command(cmd):
    return shutil.which(cmd) is not None

HAS_QE = has_command("pw.x")
HAS_LAMMPS = has_command("lmp")
HAS_PACEMAKER = has_command("pace_train")

IS_CI_MODE = os.environ.get("IS_CI_MODE", "False").lower() == "true"
if not (HAS_QE and HAS_LAMMPS and HAS_PACEMAKER):
    print("Missing external tools. Forcing CI Mode.")
    IS_CI_MODE = True

WORKDIR = Path("workdirs/02_interface")
if WORKDIR.exists():
    shutil.rmtree(WORKDIR)
WORKDIR.mkdir(parents=True, exist_ok=True)



## 2. Constructing the Interface

We manually create the scenario: A small Fe-Pt cluster deposited on an MgO(001) surface.

- **Substrate**: MgO(001) slab.
- **Adsorbate**: Fe/Pt atoms.



In [ ]:
def create_interface_structure():
    # 1. Create MgO Slab
    mgo_bulk = bulk("MgO", "rocksalt", a=4.21)
    # 2x2 surface, 3 layers thick, 10A vacuum
    slab = surface(mgo_bulk, (1, 0, 0), 3, vacuum=10.0)
    slab = slab.repeat((2, 2, 1))
    
    # 2. Add Fe and Pt atoms
    # We place them at specific sites to challenge the potential
    # add_adsorbate places atom at (x, y) relative to surface unit cell, at height 'height'
    
    # Place Fe on top of an Oxygen atom (approximate position)
    add_adsorbate(slab, "Fe", height=2.0, position=(2.1, 2.1))
    
    # Place Pt nearby
    add_adsorbate(slab, "Pt", height=2.2, position=(4.2, 2.1))
    
    slab.info["type"] = "interface"
    slab.info["generator"] = "ManualNotebook"
    
    return Structure.from_ase(slab)

interface_structure = create_interface_structure()
print(f"Created Interface: {len(interface_structure.positions)} atoms")
# view(interface_structure.to_ase()) # Uncomment to visualize



## 3. Configuring the Orchestrator

We need the Orchestrator to "focus" on this structure.
Since the standard `AdaptiveGenerator` creates random bulk/surfaces, we will **patch** it
to return our specific interface structure. This is a common pattern for "Guided Learning".



In [ ]:
# Define a custom generator function
def custom_generator_func(self, n_structures: int, cycle: int = 0, metrics = None):
    print(f"Custom Generator: Yielding interface structure for cycle {cycle}")
    # We only yield the interface once
    yield interface_structure

def create_config(workdir: Path, is_ci: bool) -> GlobalConfig:
    if is_ci:
        return GlobalConfig(
            workdir=workdir,
            max_cycles=1, # Just learn this one structure
            components={
                "generator": MockGeneratorConfig(
                    name=GeneratorType.MOCK,
                    n_structures=1,
                    cell_size=10.0,
                    n_atoms=len(interface_structure.positions),
                    atomic_numbers=interface_structure.atomic_numbers
                ),
                "oracle": MockOracleConfig(name=OracleType.MOCK),
                "trainer": MockTrainerConfig(name=TrainerType.MOCK),
                "dynamics": MockDynamicsConfig(
                    name=DynamicsType.MOCK,
                    selection_rate=1.0, # Select everything
                    simulated_uncertainty=5.0
                ),
                "validator": MockValidatorConfig(name=ValidatorType.MOCK)
            }
        )
    else:
        return GlobalConfig(
            workdir=workdir,
            max_cycles=1,
            components={
                "generator": AdaptiveGeneratorConfig(
                    name=GeneratorType.ADAPTIVE,
                    n_structures=1,
                    element="MgO", # Placeholder
                    crystal_structure="rocksalt", # Placeholder
                    policy_ratios={"cycle0_bulk": 0.0, "cycle0_surface": 0.0}
                ),
                "oracle": QEOracleConfig(
                    name=OracleType.QE,
                    ecutwfc=50.0,
                    ecutrho=400.0,
                    kspacing=0.04,
                    # We need pseudos for all 4 elements
                    pseudopotentials={
                        "Mg": "Mg.pbe-n-kjpaw_psl.1.0.0.UPF",
                        "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF",
                        "Fe": "Fe.pbe-spn-kjpaw_psl.1.0.0.UPF",
                        "Pt": "Pt.pbe-n-kjpaw_psl.1.0.0.UPF"
                    }
                ),
                "trainer": PacemakerTrainerConfig(
                    name=TrainerType.PACEMAKER,
                    cutoff=5.0,
                    max_num_epochs=50,
                    # Start from the potentials trained in Tutorial 01?
                    # For simplicity, we might start fresh or assume 'initial_potential' path
                    # initial_potential="workdirs/01_fept/cycle_02/potential.yace" 
                ),
                "dynamics": LAMMPSDynamicsConfig(
                    name=DynamicsType.LAMMPS,
                    n_steps=1000,
                    uncertainty_threshold=10.0
                ),
                "validator": StandardValidatorConfig(
                    name=ValidatorType.STANDARD
                )
            }
        )

config = create_config(WORKDIR, IS_CI_MODE)



### Run Orchestrator with Custom Generator

We patch `mlip_autopipec.components.generator.adaptive.AdaptiveGenerator.generate` (or MockGenerator)
to inject our structure.



In [ ]:
target_class = "mlip_autopipec.components.generator.adaptive.AdaptiveGenerator"
if IS_CI_MODE:
    target_class = "mlip_autopipec.components.generator.mock.MockGenerator"

# We patch the 'generate' method of the class
with patch(f"{target_class}.generate", custom_generator_func):
    orchestrator = Orchestrator(config)
    orchestrator.run()

print("Interface Learning Complete.")



## 4. Validation

Check if the structure was added to the dataset and labeled.



In [ ]:
from mlip_autopipec.core.dataset import Dataset

dataset_path = WORKDIR / "dataset.jsonl"
if IS_CI_MODE:
    assert dataset_path.exists()
else:
    if dataset_path.exists():
        ds = Dataset(dataset_path, root_dir=WORKDIR)
        count = 0
        for s in ds:
            count += 1
            # Check if it resembles our interface (approx number of atoms)
            # Validation might differ slightly due to mocked atoms in CI
            pass
        print(f"Dataset contains {count} structures.")
        assert count >= 1
    else:
        print("Dataset not found!")
